In [113]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import seaborn as sns
from Handling_Outliers import Handling_Outliers
from sklearn.neural_network import MLPClassifier


In [114]:
df_train, df_test = pd.read_csv('train.csv'), pd.read_csv('test.csv')

In [115]:
df_0 = df_train[df_train['loan_status'] == 0].sample(n=8350, random_state=42)
df_1 = df_train[df_train['loan_status'] == 1]

In [116]:
df_train = pd.concat([df_0, df_1])

In [117]:
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

In [118]:
df_sub = pd.read_csv('sample_submission.csv')

In [119]:
lb = LabelEncoder()
ss = StandardScaler()

In [120]:
def to_lb(arr):
    for i in arr:
        df_train[f'{i}'] = lb.fit_transform(df_train[f'{i}'])
    for k in arr:
        df_test[f'{k}'] = lb.fit_transform(df_test[f'{k}'])

In [121]:
to_lb(['person_home_ownership','loan_intent','loan_grade','cb_person_default_on_file'])

In [122]:
staying_columns = df_train.corr()['loan_status'][df_train.corr()['loan_status'] > 0.3].index.to_list()

In [123]:
df_train.drop(['id', 'person_age', 'person_emp_length', 'loan_intent', 'cb_person_cred_hist_length', 'cb_person_default_on_file'], axis=1, inplace=True)
df_test.drop(['id', 'person_age', 'person_emp_length', 'loan_intent', 'cb_person_cred_hist_length', 'cb_person_default_on_file'], axis=1, inplace=True)

In [124]:
# df_train = df_train[staying_columns]
# df_test = df_test[staying_columns[:-1]]

In [125]:
column_list = [i for i in df_train.columns][:-1]

In [126]:
for i in range(len(df_train.columns)):
    Handling_Outliers(df_train, df_train.columns)
for i in range(len(df_test.columns)):
    Handling_Outliers(df_test, df_test.columns)

In [127]:
X_train = df_train.drop('loan_status', axis=1)
y_train = df_train['loan_status']

In [128]:
model = MLPClassifier(hidden_layer_sizes=(128, 64, 32),
                        activation='relu', 
                        solver='adam', 
                        alpha=1e-4, 
                        batch_size=256, 
                        learning_rate_init=1e-3, 
                        max_iter=200, 
                        random_state=42)

In [129]:
model.fit(X_train, y_train)

MLPClassifier(batch_size=256, hidden_layer_sizes=(128, 64, 32), random_state=42)

In [130]:
y_pred = model.predict(df_test)

In [131]:
submission = pd.DataFrame({
    'id': df_sub['id'],  
    'loan_status': y_pred
})

In [132]:
submission.to_csv('submission14.csv', index=False)

In [133]:
submission

,id,loan_status
0,58645,1.0
1,58646,0.0
2,58647,1.0
3,58648,0.0
4,58649,0.0
...,...,...
39093,97738,1.0
39094,97739,0.0
39095,97740,1.0
39096,97741,1.0
